In [2]:
import json
import random

# -----------------------------
# 1) VOCABULAIRE — VÉGÉTATION
# -----------------------------
VEG_LOW = [
    "végétation clairsemée",
    "activité photosynthétique faible",
    "croissance végétative limitée",
    "biomasse réduite",
    "couvert végétal faible",
    "plantes peu développées",
    "faible densité de végétation",
    "développement végétatif modeste",
]

VEG_MED = [
    "végétation modérée",
    "développement végétatif correct",
    "activité photosynthétique stable",
    "biomasse intermédiaire",
    "croissance végétative convenable",
    "couvert végétal homogène",
    "dynamique végétale moyenne",
    "stade végétatif intermédiaire",
]

VEG_HIGH = [
    "végétation dense et vigoureuse",
    "biomasse élevée",
    "croissance active et homogène",
    "couvert végétal abondant",
    "activité photosynthétique intense",
    "canopée bien développée",
    "forte densité de végétation",
    "croissance végétative soutenue",
]

# -----------------------------
# 2) VOCABULAIRE — HUMIDITÉ
# -----------------------------
HUM_LOW = [
    "stress hydrique probable",
    "faible disponibilité en eau",
    "sol sec ou peu hydraté",
    "indicateurs d’humidité faibles",
    "teneur en eau insuffisante",
    "condition hydrique dégradée",
]

HUM_MED = [
    "conditions hydriques équilibrées",
    "humidité modérée",
    "teneur en eau correcte",
    "absence de stress hydrique notable",
    "niveau d’humidité normal",
    "sol globalement équilibré",
]

HUM_HIGH = [
    "bonne disponibilité hydrique",
    "couvert bien hydraté",
    "humidité élevée",
    "conditions hydriques favorables",
    "teneur en eau abondante",
    "aucun stress hydrique apparent",
]

# -----------------------------
# 3) VOCABULAIRE — SOL / VÉGÉTATION
# -----------------------------
SOIL_SOIL = [
    "présence notable de sol nu",
    "zones largement exposées",
    "faible recouvrement végétal",
    "sol visible sur une grande partie de la scène",
]

SOIL_MIX = [
    "mélange équilibré sol/végétation",
    "hétérogénéité du couvert",
    "alternance entre zones végétalisées et sol nu",
    "structure intermédiaire du couvert",
]

SOIL_VEG = [
    "végétation dominante",
    "sol très peu visible",
    "canopée couvrante",
    "recouvrement végétal important",
]

# -----------------------------
# 4) ADD-ONS EXPERTS
# -----------------------------
EXTRAS = [
    "Les valeurs spectrales sont cohérentes avec la saison en cours.",
    "La scène présente une distribution spatiale relativement homogène.",
    "Les indices ne montrent pas de signes de stress sévère.",
    "Le comportement spectral reste conforme à un cycle cultural normal.",
    "Quelques variations locales peuvent exister mais restent limitées.",
    "La signature globale est représentative d’un couvert en évolution.",
    "Les conditions observées correspondent à une phase végétative typique.",
]

# Phrases techniques optionnelles
TECH = [
    "Les contrastes NDVI/EVI renforcent l’interprétation générale.",
    "L'interaction entre bandes rouges et proche infrarouge confirme ces tendances.",
    "La cohérence NDWI–NDVI suggère une relation stable entre eau et biomasse.",
    "Les signaux spectraux montrent une bonne séparation sol–végétation.",
    "Les valeurs SAVI indiquent un comportement attendu du couvert.",
]

# -----------------------------
# 5) STRUCTURES DE PHRASES
# -----------------------------
STRUCTURES = [
    "{veg_cap}. {humidity}. {soil}. {extra}",
    "{veg_cap}. Concernant l’humidité, {humidity}. {soil}. {extra}",
    "On observe {veg}. Par ailleurs, {humidity}. Enfin, {soil}. {extra}",
    "L’analyse multi-indices suggère {veg}. Niveau hydrique : {humidity}. {soil}. {extra}",
    "Les données spectrales indiquent {veg}. {soil}. D’un point de vue hydrique : {humidity}. {extra}",
    "La scène révèle {veg}, avec {humidity}. Dans l’ensemble : {soil}. {extra}",
]

# --------------------------------------------------------
# 6) Classification par seuils
# --------------------------------------------------------
def level_ndvi(x):
    if x < 0.2: return VEG_LOW
    if x < 0.5: return VEG_MED
    return VEG_HIGH

def level_ndwi(x):
    if x < 0.1: return HUM_LOW
    if x < 0.3: return HUM_MED
    return HUM_HIGH

def level_savi(x):
    if x < 0.2: return SOIL_SOIL
    if x < 0.4: return SOIL_MIX
    return SOIL_VEG

# --------------------------------------------------------
# 7) Génération d'une caption
# --------------------------------------------------------
def generate_caption(rec):
    veg = random.choice(level_ndvi(rec["ndvi_mean"]))
    humidity = random.choice(level_ndwi(rec["ndwi_mean"]))
    soil = random.choice(level_savi(rec["savi_mean"]))

    extra = random.choice(EXTRAS)

    # 25% de chances d’ajouter une phrase technique supplémentaire
    if random.random() < 0.25:
        extra += " " + random.choice(TECH)

    template = random.choice(STRUCTURES)

    return template.format(
        veg=veg,
        veg_cap=veg.capitalize(),
        humidity=humidity,
        soil=soil.capitalize(),
        extra=extra
    )

# --------------------------------------------------------
# 8) Pipeline complet pour créer un nouveau JSONL
# --------------------------------------------------------
IN_FILE = "sentinel_indices.jsonl"
OUT_FILE = "sentinel_indices_v2.jsonl"

with open(IN_FILE, "r") as fin, open(OUT_FILE, "w") as fout:
    for line in fin:
        rec = json.loads(line)
        rec["caption"] = generate_caption(rec)
        fout.write(json.dumps(rec) + "\n")

print("JSONL Version 2 généré avec succès →", OUT_FILE)

JSONL Version 2 généré avec succès → sentinel_indices_v2.jsonl
